In [ ]:
import os
import time
import json
import numpy as np
import bmtools.exact.moments as mom
import bmtools.relaxations.gm_relaxations as gmr
import bmtools.utils as utils

In [ ]:
base_dir = os.path.dirname(os.getcwd())
model_dir = os.path.join(base_dir, 'data', 'gaussian-bmr')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [ ]:
params = {
    'seed': 201702,
    'n_unit': 28,
    'n_sys': 10,
    'w_scale_1': 6.,
    'w_scale_2': 2.,
    'b_scale': 0.1,
}
with open(os.path.join(model_dir, 'params.json'), 'w') as f:
    json.dump(params, f, indent=True)

In [ ]:
rng = np.random.RandomState(params['seed'])
for s in range(params['n_sys']):
    print('Starting system {0}...'.format(s + 1))
    basis = utils.random_orthogonal_matrix(params['n_unit'], rng)
    eigvals = params['w_scale_1'] * np.tanh(rng.normal(size=params['n_unit']) * params['w_scale_2'])
    biases = rng.normal(size=params['n_unit']) * params['b_scale']
    weights = (basis * eigvals).dot(basis.T)
    weights -= np.diag(weights.diagonal())
    relaxation = gmr.IsotropicCovarianceGMRelaxation(weights, biases, True)
    log_norm_const_x, expc_x, covar_x = relaxation.moments_x(True, 4)
    save_path = os.path.join(model_dir, 'params_and_moms_{0}.npz'.format(s + 1))
    np.savez_compressed(save_path, weights=weights, biases=biases, 
                        log_norm_const_x=log_norm_const_x, expc_x=expc_x, covar_x=covar_x)
    print('...completed system {0}.'.format(s + 1))